In [12]:
from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud

In [13]:
filterwarnings ('ignore') # bazı warningleri ignore etme 
pd.set_option('display.max_columns', None)   # bütün sütunları göster 
pd.set_option('display.width', 200)   # max genişlik ayarı
pd.set_option('display.float_format', lambda x: '%.2f' % x) # virgülden sonra iki basamak göster

# METİN ÖN İŞLEME # 

Veri setinde problemli olarak kabul edilecek durumlardan kurtulunur. Bir metinde çıkarılması gereken yapılar ortadan kaldırılır.

In [14]:
df = pd.read_csv("amazon_reviews.csv") # veri setini okuma

In [15]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00,best deal around,1373673600,2013-07-13,513,0,0


# bu veri seti bir ürüne yapılan yorumları ifade eder

- reviewerID : yorum yapan kişiler
- asin : ürün 
- helpful : yorumların faydalı olup olmadığına ilişkin değişken
- reviewText : yorumlar, *odaklanacağımız değişken*
- overall : ürüne verilen puan 
- summary : yorumların özeti 
- helpful_yes,total_vote : helpful'un açılmış hali


In [16]:
df['reviewText']  

0                                              No issues.
1       Purchased this for my device, it worked as adv...
2       it works as expected. I should have sprung for...
3       This think has worked out great.Had a diff. br...
4       Bought it with Retail Packaging, arrived legit...
                              ...                        
4910    I bought this Sandisk 16GB Class 10 to use wit...
4911    Used this for extending the capabilities of my...
4912    Great card that is very fast and reliable. It ...
4913    Good amount of space for the stuff I want to d...
4914    I've heard bad things about this 64gb Micro SD...
Name: reviewText, Length: 4915, dtype: object

In [18]:
# verilerde bazı hafrkler büyük bazı harfler küçük, belirli bir standarta koyulmalı
# Normalizing Case Folding
###

df ['reviewText'] = df['reviewText'].str.lower()
# bütün kelimeler küçük harfle yazdır çünkü aynı kelime büyük harfle başlasaydı o kelime metinde aranmak istenince bulunamazdı 
df['reviewText']

0                                              no issues.
1       purchased this for my device, it worked as adv...
2       it works as expected. i should have sprung for...
3       this think has worked out great.had a diff. br...
4       bought it with retail packaging, arrived legit...
                              ...                        
4910    i bought this sandisk 16gb class 10 to use wit...
4911    used this for extending the capabilities of my...
4912    great card that is very fast and reliable. it ...
4913    good amount of space for the stuff i want to d...
4914    i've heard bad things about this 64gb micro sd...
Name: reviewText, Length: 4915, dtype: object

# metinde işimize yarmayacak, bizim için bir anlam içermeyen ifadeler silinmeli

# regular expression (regex)  
metinde belirli bir örüntüyü yakalayıp üzerinde çalışmak için kullanılan bir yapıdır

In [43]:
# REMOVE PUNCTUATIONS # noktalam işaretinin az çok olması bir anlam ifade etmez 

import re # regex
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub("[^\w\s]","", str(x)))
# metinde noktalama işareti yerine boşluk eklenir 

df['reviewText']

0                                               no issues
1       purchased this for my device it worked as adve...
2       it works as expected i should have sprung for ...
3       this think has worked out greathad a diff bran...
4       bought it with retail packaging arrived legit ...
                              ...                        
4910    i bought this sandisk 16gb class 10 to use wit...
4911    used this for extending the capabilities of my...
4912    great card that is very fast and reliable it c...
4913    good amount of space for the stuff i want to d...
4914    ive heard bad things about this 64gb micro sd ...
Name: reviewText, Length: 4915, dtype: object

In [45]:
# REMOVE NUMBERS # bazı metinlerde rakamlar anlam ifade edebilir, bu veri için gerekli değil

df['reviewText'] = df['reviewText'].fillna('').apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
df["reviewText"] 

0                                               no issues
1       purchased this for my device it worked as adve...
2       it works as expected i should have sprung for ...
3       this think has worked out greathad a diff bran...
4       bought it with retail packaging arrived legit ...
                              ...                        
4910    i bought this sandisk gb class  to use with my...
4911    used this for extending the capabilities of my...
4912    great card that is very fast and reliable it c...
4913    good amount of space for the stuff i want to d...
4914    ive heard bad things about this gb micro sd ca...
Name: reviewText, Length: 4915, dtype: object

In [46]:
# REMOVE STOP WORDS # metinde herhangi bir anlamı olmayan ama dilde kullanılan yaygın kelimeler: bağlaçlar, zamirler vb. (for, is, of) bu gibi ifadelerin dilde ölçümsel bir anlamı yoktur
import nltk 
nltk.download('stopwords') # nltk kütüphanesi içinde ingilizce dili için stopwords listesi indirilir, türkçe dili içinde benzer çalışmlar 

sw = stopwords.words('english') # bu kelimler sw içerisine aktarılır 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\havva\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [47]:
sw # kendi setimizi de oluşturabiliriz

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [48]:
# bütün satırları gezip bütün kelimelere ulaşmalıyız, apply df içinde bütün satırları gezme için 

# REMOVE STOPWORD # 

df["reviewText"] = df["reviewText"].apply(lambda x: " ".join(x for x in str(x).split() if x not in sw))

# x : gelen satır ifadesi
# gelen satır stringe çevrilir ve boşluğa göre split edilerek ayırlır, elde edilen kelime sw içerisinde değilse o kelimeyi al ve kaldır 
df["reviewText"]

0                                                  issues
1       purchased device worked advertised never much ...
2       works expected sprung higher capacity think ma...
3       think worked greathad diff bran gb card went s...
4       bought retail packaging arrived legit orange e...
                              ...                        
4910    bought sandisk gb class use htc inspire months...
4911    used extending capabilities samsung galaxy not...
4912    great card fast reliable comes optional adapte...
4913          good amount space stuff want fits gopro say
4914    ive heard bad things gb micro sd card crapping...
Name: reviewText, Length: 4915, dtype: object

In [50]:
# RARE WORDS # nadir olan kelimeleri çıkarabiliz veri setinden 

# bütün kelimlerin kaçar defa geçtiğini bulup belli bir sayıya gmre nadir geçenleri kaldırabiliriz

temp_df = pd.Series(' '.join(df['reviewText']).split()).value_counts() # bütün kelimleri saydırıyoruz ve geçici dataframe aktarıyoruz
temp_df

card          4603
gb            1725
phone         1685
works         1559
great         1445
              ... 
sharper          1
cheapos          1
gate             1
measuring        1
stealwould       1
Name: count, Length: 9701, dtype: int64

In [51]:
drops = temp_df[temp_df <= 1] # 1den az frekanslıları seçip dropsa atıyoruz
drops

conclusionmy       1
mbsbuy             1
flimsier           1
disappointedtry    1
priceokay          1
                  ..
sharper            1
cheapos            1
gate               1
measuring          1
stealwould         1
Name: count, Length: 5612, dtype: int64

In [52]:
df["reviewText"] = df["reviewText"].apply(lambda x: " ".join(x for x in str(x).split() if x not in drops)) # dropsta yer alanlar boşluk ile yer değiştirilir
df["reviewText"]

0                                                  issues
1       purchased device worked advertised never much ...
2       works expected higher capacity think made bit ...
3       think worked gb card went south one held prett...
4       bought retail packaging arrived legit envelope...
                              ...                        
4910    bought sandisk gb class use htc inspire months...
4911    used capabilities samsung galaxy note greatly ...
4912    great card fast reliable comes optional adapte...
4913          good amount space stuff want fits gopro say
4914    ive heard bad things gb micro sd card crapping...
Name: reviewText, Length: 4915, dtype: object

In [54]:
# TOKENIZATION # cümleleri tokenlarına ayırma yani parçalama, birimleştirme işlemini programatik hale getirmek 

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\havva\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [55]:
df["reviewText"].apply(lambda x: TextBlob(x).words).head() # TextBlob metodu çalıştıırıldıktan sonra kelimler gelsin 

0                                             [issues]
1    [purchased, device, worked, advertised, never,...
2    [works, expected, higher, capacity, think, mad...
3    [think, worked, gb, card, went, south, one, he...
4    [bought, retail, packaging, arrived, legit, en...
Name: reviewText, dtype: object

In [56]:
# LEMMATIZATION # kelimleri köklerine ayırmak / literatürde stemming işlemi de kullanılır köklerine ayırmak için 
# en: kelimelerin sonlarındaki -s takılarını ortadan kaldırma , tr: gözlük, gözlükçü => göz

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\havva\AppData\Roaming\nltk_data...


True

In [60]:
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) # bütün kelimleri köklerine ayırır
df['reviewText']

0                                                   issue
1       purchased device worked advertised never much ...
2       work expected higher capacity think made bit e...
3       think worked gb card went south one held prett...
4       bought retail packaging arrived legit envelope...
                              ...                        
4910    bought sandisk gb class use htc inspire month ...
4911    used capability samsung galaxy note greatly ex...
4912    great card fast reliable come optional adapter...
4913           good amount space stuff want fit gopro say
4914    ive heard bad thing gb micro sd card crapping ...
Name: reviewText, Length: 4915, dtype: object

# METİN GÖRSELLEŞTİRME # 